In [1]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
import os
from dotenv import load_dotenv

In [2]:
# loading .env
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0)

In [19]:
! pip list | findstr langchain

langchain                                0.1.16
langchain-chroma                         0.1.0
langchain-community                      0.0.34
langchain-core                           0.1.45
langchain-openai                         0.1.3
langchain-text-splitters                 0.0.1


In [4]:
# from langchain_core.prompts import ChatPromptTemplate
# prompt = ChatPromptTemplate.from_messages([
#     ("system", """
#      You are familiar with this person Manish. Here are some information that you know about him:
#      Name: Manish
#      Occupation: revenue operations intern at SAP
#      job scope: sales support via excel consolidation
#      reports to: Magdalene Usada (also known as Maggy)
#      colleagues: ["Drae Ramirez", "Daphne Koh", "Lindsey", "Monte Reynaldo", "Elizabeth"]
#      STAR batch intern: 6
#      STAR batch: 2024
#      STAR batch mentor: Irene and Jean
#      STAR batch colleagues: ['Saniya', 'Jia Ning', "Ren Yi", "Saieesh", "Jean Tay", "Boyi", "Ting Fung", "Khai"]
#      Onboarding date: Dec 2023
#      Official start date: Jan 2024
#      year of graduation and end date: May 2025
#      favorite movie: Om Shanti Om
#      languages: benagli, english
#      favorite book: The Millionare Next Door
#      favorite song: House of the rising sun
#      favorite TV series: Breaking Bad
#      next rotations of interest: generative AI engineer, Data Analyst Signavio, Datahub Digital Intelligence Analyst
#      Fan: Lex Friedman
#      Age: 24
#      Friends: ["Samuel, Barnabas"]
#      Interests and hobbies: coding, reading, working out
#      Favorite food: Ramen
#      guilty pleasure: Ramen
#      pet peeves: people who don't listen
#      Favorite color: Blue
#      Aspiration: Machine Learning Engineer at SAP
#      Lives in: Singapore (Sengkang)
#      Gender and sex: Male
#      Race: Bengali
#      Religion: Free Thinker
#      Height: 175 cm
#      Weight: 68 kg
#      Course of study: Data Science and Analytics
#      Specialization: Operations Research
#      Motto: "Do what you love, love what you do"
#      National Service Status: Completed (Singapore Police Force, Transport Command, 1st Sergeant)
#      Reservist Status: 2nd Cycle
#      If there is anything that you don't know about him, you can ask him.
#      When prompted for an answer to a question that you don't know, you can say "I don't know".
#      """),
#     ("user", "{input}")
# ])

In [5]:
# from langchain_core.output_parsers import StrOutputParser

# output_parser = StrOutputParser()

In [6]:
# chain = prompt | llm | output_parser

In [7]:
# def handle_user_input():
    
#     while True:
#         input_str = input("Enter your message: ")
#         response = chain.invoke({"input": input_str})
#         print(response)
#         if(input_str == "exit"):
#             break

In [8]:
# print("Manish chatbot is ready. exit the chatbot by typing 'exit'")
# handle_user_input()

Alt approach

In [9]:
from langchain.document_loaders import TextLoader

path = "../data/general_data.txt"
loader = TextLoader(path)
documents = loader.load()

In [10]:
# document chunking
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

In [11]:
# setting up the vector store
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
db = Chroma.from_documents(chunks, OpenAIEmbeddings())

In [12]:
# retreival
retriever = db.as_retriever()

In [13]:
# augmented
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

In [14]:
# generation
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)
query = "What is Manish's favorite food?"
rag_chain.invoke(query)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


"Manish's favorite food is Ramen."

In [15]:
def handle_user_input():
    
    while True:
        input_str = input("Enter your message: ")
        if(input_str == "exit"):
            break
        response = rag_chain.invoke(input_str)
        print(response)

In [16]:
print("Manish chatbot is ready. Exit the chatbot by typing 'exit'")
handle_user_input()

Manish chatbot is ready. Exit the chatbot by typing 'exit'
